In [1]:
import networkx as nx

from regraph import Rule, plot_rule
from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [8]:
# initialize the neo4j driver, wrappped into Neo4jGraph object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [14]:
# here we clear the hierarchy
h.clear()
h.drop_all_constraints()

## Adding a graph to the hierarchy

In [15]:
h.add_graph('graphB')
b = h.access_graph('graphB')
nodes = [
    ("a", {"name": "white_circle"}),
    ("b", {"name": "black_circle"}),
    ("c", {"name": "square"})
]
edges = [
    ("a", "b"),
    ("b", "c")
]
b.add_nodes_from(nodes)
b.add_edges_from(edges)

In [16]:
h.add_graph('graphC')
c = h.access_graph('graphC')
nodes = [
    ("a", {"name": "white_square"}),
    ("b", {"name": "black_square"}),
    ("c", {"name": "circle"})
]
edges = [
    ("a", "b"),
    ("c", "b"),
    ("c", "c")
]
c.add_nodes_from(nodes)
c.add_edges_from(edges)

In [17]:
h.add_graph('graphD')
d = h.access_graph('graphD')
nodes = [
    ("a", {"name": "circle"}),
    ("b", {"name": "square"})
]
edges = [
    ("a", "b"),
    ("a", "a"),
    ("b", "b")
]
d.add_nodes_from(nodes)
d.add_edges_from(edges)

## Typing

In [18]:
mappingBD = {
    "a":"a",
    "b":"a",
    "c":"b"
}
h.add_typing('graphB', 'graphD', mappingBD)
mappingCD = {
    "a":"b",
    "b":"b",
    "c":"a"
}
h.add_typing('graphC', 'graphD', mappingCD)